In [0]:
dbutils.widgets.text("p_environment", "")
dbutils.widgets.text("p_instance_name", "")
dbutils.widgets.text("p_product_id", "")
dbutils.widgets.text("p_client_id", "")
dbutils.widgets.text("p_facility_id", "")
dbutils.widgets.text("p_step_type", "")
dbutils.widgets.text("p_data_source_type", "")
dbutils.widgets.text("p_data_source_id", "")

from pyspark.sql.functions import col, lit

In [0]:
%run "./DataFabricCommonFunctions"

In [0]:
v_environment = dbutils.widgets.get("p_environment")
v_data_source_type = dbutils.widgets.get("p_data_source_type")
v_data_source_id = dbutils.widgets.get("p_data_source_id")
v_instance_name = dbutils.widgets.get("p_instance_name")
v_product_id = dbutils.widgets.get("p_product_id")
v_client_id = dbutils.widgets.get("p_client_id")
v_facility_id = dbutils.widgets.get("p_facility_id")
v_step_type = dbutils.widgets.get("p_step_type")
v_df_process_status_facility_detail = execute_dbconfig_sql_query("SELECT * FROM cfg.processstatusfacilitydetail", v_environment)
v_df_process_status_facility = execute_dbconfig_sql_query("SELECT * FROM cfg.processstatusfacility", v_environment)
v_df_data_source = execute_dbconfig_sql_query("SELECT * FROM cfg.datasource", v_environment)
v_data_source_id = v_df_data_source.filter(col("DataSourceType") == f"{v_data_source_type}").first()["DataSourceId"]

In [0]:
if v_data_source_type == "Instance":
    v_process_status_facility_count = v_df_process_status_facility.filter(col("InternalProductId") == v_product_id).filter(col("SourceInstanceServerName") == v_instance_name).filter(col("StepType") == v_step_type).filter(col("DataSourceId") == v_data_source_id).count()

    v_process_status_facility_detail_count = v_df_process_status_facility_detail.filter(col("InternalProductId") == v_product_id).filter(col("SourceInstanceServerName") == v_instance_name).filter(col("StepType") == v_step_type).filter(col("DataSourceId") == v_data_source_id).select("InternalProductId", "InternalClientIds", "InternalFacilityId", "DataSourceId", "StepType", "StepName", "SourceInstanceServerName").distinct().count()

    if v_process_status_facility_count == 0:
        execute_dbconfig_stored_procedure(
        f"""
        EXEC CFG.InsertProcessStatusFacility @InternalProductId = {v_product_id}, @InternalClientId = NULL, @InternalFacilityId = NULL, @SourceInstanceServerName = '{v_instance_name}', @IsInstance = 1, @StepType = {v_step_type}, @DataSourceId = 46
        """,v_environment
        )

    execute_dbconfig_stored_procedure(
    f"""
    EXEC CFG.InsertProcessStatusFacilityDetail @InternalProductId = {v_product_id}, @SourceInstanceServerName = '{v_instance_name}'
    """,v_environment
    )

elif v_data_source_type == "Product":
    if not v_facility_id:
        v_process_status_facility_count = v_df_process_status_facility.filter(col("InternalProductId") == v_product_id).filter(col("InternalClientIds") == v_client_id).filter(col("InternalFacilityId") == 0).filter(col("StepType") == v_step_type).filter(col("DataSourceId") == 47).count()

        v_process_status_facility_detail_count = v_df_process_status_facility_detail.filter(col("InternalProductId") == v_product_id).filter(col("InternalClientIds") == v_client_id).filter(col("InternalFacilityId") == 0).filter(col("StepType") == v_step_type).filter(col("DataSourceId") == 47).select("InternalProductId", "InternalClientIds", "InternalFacilityId", "DataSourceId", "StepType", "StepName", "SourceInstanceServerName").distinct().count()

        if v_process_status_facility_count == 0:
            execute_dbconfig_stored_procedure(
            f"""
            EXEC CFG.InsertProcessStatusFacility @InternalProductId = {v_product_id}, @InternalClientId = {v_client_id}, @InternalFacilityId = NULL, @SourceInstanceServerName = '{v_instance_name}', @IsInstance = 0, @StepType = {v_step_type}, @DataSourceId = 47
            """,v_environment
            )
        
        execute_dbconfig_stored_procedure(
        f"""
        EXEC CFG.InsertProcessStatusFacilityDetail @InternalProductId = {v_product_id}, @InternalClientId = {v_client_id}, @SourceInstanceServerName = '{v_instance_name}'
        """,v_environment
        )
    else:
        v_process_status_facility_count = v_df_process_status_facility.filter(col("InternalProductId") == v_product_id).filter(col("InternalClientIds") == v_client_id).filter(col("InternalFacilityId") == v_facility_id).filter(col("StepType") == v_step_type).filter(col("DataSourceId") == 47).count()

        v_process_status_facility_detail_count = v_df_process_status_facility_detail.filter(col("InternalProductId") == v_product_id).filter(col("InternalClientIds") == v_client_id).filter(col("InternalFacilityId") == v_facility_id).filter(col("StepType") == v_step_type).filter(col("DataSourceId") == 47).select("InternalProductId", "InternalClientIds", "InternalFacilityId", "DataSourceId", "StepType", "StepName", "SourceInstanceServerName").distinct().count()

        if v_process_status_facility_count == 0:
            execute_dbconfig_stored_procedure(
            f"""
            EXEC CFG.InsertProcessStatusFacility @InternalProductId = {v_product_id}, @InternalClientId = {v_client_id}, @InternalFacilityId = {v_facility_id}, @SourceInstanceServerName = '{v_instance_name}', @IsInstance = 0, @StepType = {v_step_type}, @DataSourceId = 47
            """,v_environment
            )
            
        execute_dbconfig_stored_procedure(
        f"""
        EXEC CFG.InsertProcessStatusFacilityDetail @InternalProductId = {v_product_id}, @InternalClientId = {v_client_id}, @InternalFacilityId = {v_facility_id}, @SourceInstanceServerName = '{v_instance_name}'
        """,v_environment
        )